In [7]:
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import math
import numpy  
import pandas as pd  
import os


#Step 2. 사용자에게 검색어 키워드를 입력 받고 저장할 폴더와 파일명을 설정합니다.
print("=" *80)
print("뉴스 기사의 댓글 정보 수집하기")
print("=" *80)

query_txt = '뉴스기사댓글'
query_url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=056&aid=0010661268'
try :
    cnt = int(input('1.크롤링 할 뉴스 리뷰 건수는 몇건입니까?(기본값: 20건): '))
except ValueError :
    cnt = 20

page_cnt = math.ceil(cnt / 20)

f_dir = input("2.결과를 파일을 저장할 폴더명만 쓰세요(기본값:c:\\py_temp\\):")
if f_dir=='' :
    f_dir='c:\\py_temp\\'
    
# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, \
                                       now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get(query_url)
driver.maximize_window()
time.sleep(5)

# 현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화합니다
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('div','media_end_head_info_variety_cmtcount _COMMENT_HIDE').get_text()

print("=" *80)
search_cnt = int( result.replace(",",""))

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수 :",cnt)
print("총 페이지 수:" , page_cnt)

뉴스 기사의 댓글 정보 수집하기
1.크롤링 할 뉴스 리뷰 건수는 몇건입니까?(기본값: 20건): 10
2.결과를 파일을 저장할 폴더명만 쓰세요(기본값:c:\py_temp\):
전체 검색 결과 건수 : 505 건
실제 최종 출력 건수 : 10
총 페이지 수: 1


In [ ]:
# 사용자가 요청한 건수가 많을 경우 리뷰 더보기 버튼을 클릭합니다
# 최초 10건 수집후 댓글 더보기 버튼 클릭
# 아래 버튼을 눌러 첫 화면에 총 20건의 댓글이 나오게 만듦
driver.find_element(By.XPATH,'//*[@id="cbox_module"]/div/div[9]/a/span[1]').click()
time.sleep(3)

#Step 6. 20건 출력되어 있는 현재 페이지 리뷰와 점수 등 내용 수집
no2= []             # 리뷰 번호 
writer_id2=[]       # 리뷰 작성자 ID
review2=[]          # 리뷰 내용
write_date2=[]      # 리뷰 작성 일자
gogam_0=[]          # 공감 횟수
gogam_1=[]          # 비공감 횟수
count = 0

for a in range(1,page_cnt+1) :
    
    if a == page_cnt :
          break
    else :
        driver.find_element(By.XPATH,'//*[@id="cbox_module"]/div/div[9]/a').click() 
        time.sleep(3)
        print("%s페이지 이동 완료=================================================" %a)
        time.sleep(random.randrange(1,3))  # 3-8 초 사이에 랜덤으로 시간 선택

print('이제 리뷰 정보를 수집합니다. 잠시만 기다려 주세요~~~~~~~~')

#txt 파일에 저장하기 위해 파일 open하기
f = open(ff_name, 'a',encoding='UTF-8')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

slist = soup.find('ul', 'u_cbox_list').find_all('li')

for li in slist:
    count += 1
    print("\n")
    print("총 %s건 중 %s번째 댓글 수집 중입니다 ==================" %(cnt,count))
    
    # 리뷰 번호 저장
    no2.append(count)
    
    # 리뷰 작성자 ID 저장
    writer_id = li.find('span','u_cbox_nick').get_text()
    print("1.작성자ID:", writer_id)
    f.write("\n")
    f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count) + "\n")
    f.write("1.작성자ID:"+writer_id + "\n")
    writer_id2.append(writer_id)
    
    #리뷰 내용 저장
    try :
        review = li.find('span', class_='u_cbox_contents').get_text()
    except AttributeError :
        review='작성자에 의해 삭제된 댓글입니다'
        print("2.리뷰 :",review)
    else :
        print("2.리뷰:",review)
    f.write("2.리뷰:" + review + "\n")
    review2.append(review)

    # 리뷰 작성일자 저장
    write_date = li.find('span',class_='u_cbox_date').get_text()
    print('3.작성일자:',write_date)
    f.write("3.작성일자:" + write_date + "\n")
    write_date2.append(write_date)

    # 공감 / 비공감 횟수 저장
    gogam = li.find('div', class_='u_cbox_recomm_set').find_all('em')
    
    try :
        g_gogam = gogam[0].text
        print('4.공감:',g_gogam)
    except IndexError :
        g_gogam = '0'
        print('4.공감 :',g_gogam)
    f.write("4.공감:" + g_gogam + "\n")
    gogam_0.append(g_gogam)

    try :
        b_gogam = gogam[1].text
        print('5.비공감:',b_gogam) 
    except IndexError :
        b_gogam = '0'
        print('5.비공감 :',b_gogam)
    f.write("5.비공감:" + b_gogam + "\n")
    gogam_1.append(b_gogam)
      
    time.sleep(0.2)      

    if count == cnt :
         break

In [ ]:
# 출력 결과를 표(데이터 프레임) 형태로 만들기
news_review = pd.DataFrame()
news_review['번호'] = no2
news_review['작성자ID'] = pd.Series(writer_id2)
news_review['리뷰내용'] = pd.Series(review2)
news_review['작성일자'] = pd.Series(write_date2)
news_review['공감횟수'] = pd.Series(gogam_0)
news_review['비공감횟수'] = pd.Series(gogam_0)

# csv 형태로 저장하기
news_review.to_csv(fc_name,encoding="utf-8-sig",index=False)
print(" csv 파일 저장 경로: %s" %fc_name) 

# 엑셀 형태로 저장하기
news_review.to_excel(fx_name , index=False , encoding="UTF-8" , engine='openpyxl')
print(" xls 파일 저장 경로: %s" %fx_name) 